In [1]:
import os, sys, math, io
import numpy as np
import pandas as pd
import multiprocessing as mp
import bson
import struct

%matplotlib inline
import matplotlib.pyplot as plt

import keras
from keras.preprocessing.image import load_img, img_to_array
import tensorflow as tf

from collections import defaultdict
from tqdm import *

Using TensorFlow backend.


In [2]:
# First load the lookup tables from the CSV files

categories_df = pd.read_csv("categories.csv", index_col=0)

## method borrord from part 01 of this notebook
def make_category_tables():
    cat2idx = {}
    idx2cat = {}
    for ir in categories_df.itertuples():
        category_id = ir[0]
        category_idx = ir[4]
        cat2idx[category_id] = category_idx
        idx2cat[category_idx] = category_id
    return cat2idx, idx2cat
##################################

cat2idx, idx2cat = make_category_tables()

train_offsets_df = pd.read_csv("train_offsets.csv", index_col=0)
train_images_df = pd.read_csv("train_images.csv", index_col=0)
val_images_df = pd.read_csv("val_images.csv", index_col=0)

test_offsets_df = pd.read_csv("test_offsets.csv", index_col=0)
test_images_df = pd.read_csv("test_images.csv", index_col=0)

/home/cvpr/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


The Keras generator is implemented by the BSONIterator class. It creates batches of images (and their one-hot encoded labels) directly from the BSON file. It can be used with multiple workers.

Note: For fastest results, put the train.bson and test.bson files on a fast drive (SSD).

See also the code in: https://github.com/fchollet/keras/blob/master/keras/preprocessing/image.py

In [3]:
from keras.preprocessing.image import Iterator
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K

class BSONIterator(Iterator):
    def __init__(self, bson_file, images_df, offsets_df, num_class,
                 image_data_generator, lock, target_size=(180, 180), 
                 with_labels=True, batch_size=32, shuffle=False, seed=None):

        self.file = bson_file
        self.images_df = images_df
        self.offsets_df = offsets_df
        self.with_labels = with_labels
        self.samples = len(images_df)
        self.num_class = num_class
        self.image_data_generator = image_data_generator
        self.target_size = tuple(target_size)
        self.image_shape = self.target_size + (3,)

        print("Found %d images belonging to %d classes." % (self.samples, self.num_class))

        super(BSONIterator, self).__init__(self.samples, batch_size, shuffle, seed)
        self.lock = lock

    def _get_batches_of_transformed_samples(self, index_array):
        batch_x = np.zeros((len(index_array),) + self.image_shape, dtype=K.floatx())
        if self.with_labels:
            batch_y = np.zeros((len(batch_x), self.num_class), dtype=K.floatx())

        for i, j in enumerate(index_array):
            # Protect file and dataframe access with a lock.
            with self.lock:
                image_row = self.images_df.iloc[j]
                product_id = image_row["product_id"]
                offset_row = self.offsets_df.loc[product_id]

                # Read this product's data from the BSON file.
                self.file.seek(offset_row["offset"])
                item_data = self.file.read(offset_row["length"])

            # Grab the image from the product.
            item = bson.BSON.decode(item_data)
            img_idx = image_row["img_idx"]
            bson_img = item["imgs"][img_idx]["picture"]

            # Load the image.
            img = load_img(io.BytesIO(bson_img), target_size=self.target_size)

            # Preprocess the image.
            x = img_to_array(img)
            x = self.image_data_generator.random_transform(x)
            x = self.image_data_generator.standardize(x)

            # Add the image and the label to the batch (one-hot encoded).
            batch_x[i] = x
            if self.with_labels:
                batch_y[i, image_row["category_idx"]] = 1

        if self.with_labels:
            return batch_x, batch_y
        else:
            return batch_x

    def next(self):
        with self.lock:
            index_array = next(self.index_generator)
        return self._get_batches_of_transformed_samples(index_array)

In [4]:
import threading
lock = threading.Lock()

In [5]:
## borrowed from part 01
data_dir = "../data/"

train_bson_path = os.path.join(data_dir, "train.bson")
num_train_products = 7069896

# train_bson_path = os.path.join(data_dir, "train_example.bson")
# num_train_products = 82

test_bson_path = os.path.join(data_dir, "test.bson")
num_test_products = 1768182


In [6]:
train_bson_file = open(train_bson_path, "rb")

Create a generator for training and a generator for validation.

In [7]:
def preprocess_img(x):
    '''
        converts RGB values from [0,255] to [-1, 1]
    '''
    return ((x/255.0)-0.5)*2

In [8]:
num_classes = 5270
num_train_images = len(train_images_df)
num_val_images = len(val_images_df)
batch_size = 256

# Tip: use ImageDataGenerator for data augmentation and preprocessing.
train_datagen = ImageDataGenerator( preprocessing_function=preprocess_img,
                                    horizontal_flip=True)
train_gen = BSONIterator(train_bson_file, train_images_df, train_offsets_df, 
                         num_classes, train_datagen, lock,
                         batch_size=batch_size, shuffle=True)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_img)
val_gen = BSONIterator(train_bson_file, val_images_df, train_offsets_df,
                       num_classes, val_datagen, lock,
                       batch_size=batch_size, shuffle=True)

Found 11137481 images belonging to 5270 classes.
Found 1233812 images belonging to 5270 classes.


In [ ]:
#How fast is the generator? Create a single batch:
next(train_gen)  # warm-up
%time bx, by = next(train_gen) #bx=image_batch, by=label_batch

Does it really output images and one-hot encoded class labels? 

Note that the images are pre-processed (and augmented) and therefore may look weird.

In [ ]:
def reverse_preprocess_img(x):
    return ((x / 2.0)+0.5)*255.0

plt.imshow(bx[-1].astype(np.uint8))

In [ ]:
cat_idx = np.argmax(by[-1])
cat_id = idx2cat[cat_idx]
categories_df.loc[cat_id]

In [ ]:
%time bx, by = next(val_gen)

In [ ]:
plt.imshow(bx[-1].astype(np.uint8))

In [ ]:
cat_idx = np.argmax(by[-1])
cat_id = idx2cat[cat_idx]
categories_df.loc[cat_id]

In [ ]:
bx.shape

In [ ]:
from keras.layers import Dense
from keras.models import Model
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input

In [ ]:
base_model = InceptionResNetV2(include_top=False, pooling='avg',input_shape=(180, 180, 3))
outputs = Dense(num_classes, activation='softmax')(base_model.output)
model = Model(base_model.inputs, outputs)

In [ ]:
model.summary()

In [ ]:
for layer in model.layers:
    layer.trainable = False

In [ ]:
model.summary()

In [ ]:
## training last 20 layers 
for layer in model.layers[-20:]:
    layer.trainable = True

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from keras_tqdm import TQDMNotebookCallback

## First epoch

In [ ]:
# To train the model:
train_history = model.fit_generator(train_gen,
                    steps_per_epoch =math.ceil(train_gen.n/batch_size),
                    epochs = 1,
                    validation_data = val_gen,
                    validation_steps = math.ceil(val_gen.n/batch_size),
                    callbacks=[TQDMNotebookCallback()],
                    verbose=0, 
                    workers = 18)
model.save('Test.h5')

In [ ]:
train_history.history

## Epoch 2

In [ ]:
# To train the model:
train_history = model.fit_generator(train_gen,
                    steps_per_epoch =math.ceil(train_gen.n/batch_size),
                    epochs = 1,
                    validation_data = val_gen,
                    validation_steps = math.ceil(val_gen.n/batch_size),
                    callbacks=[TQDMNotebookCallback()],
                    verbose=0, 
                    workers = 18)
model.save('Inception_resnetv2_epoch2.h5')

In [ ]:
train_history.history

## Epoch 3

In [ ]:
train_history = model.fit_generator(train_gen,
                    steps_per_epoch =math.ceil(train_gen.n/batch_size),
                    epochs = 1,
                    validation_data = val_gen,
                    validation_steps = math.ceil(val_gen.n/batch_size),
                    callbacks=[TQDMNotebookCallback()],
                    verbose=0, 
                    workers = 18)
model.save('Inception_resnetv2_epoch3.h5')

In [ ]:
train_history.history

## Epoch 4

In [ ]:
## training last 36 layers 
for layer in model.layers[-36:]:
    layer.trainable = True

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
train_history = model.fit_generator(train_gen,
                    steps_per_epoch =math.ceil(train_gen.n/batch_size),
                    epochs = 1,
                    validation_data = val_gen,
                    validation_steps = math.ceil(val_gen.n/batch_size),
                    callbacks=[TQDMNotebookCallback()],
                    verbose=0, 
                    workers = 16)
model.save('Inception_resnetv2_epoch4.h5')

In [ ]:
train_history.history

## Epoch 5

In [ ]:
from keras.models import load_model
from keras_tqdm import TQDMNotebookCallback
model = load_model('Inception_resnetv2_epoch4.h5')

In [ ]:
## training last 52 layers 
for layer in model.layers[-52:]:
    layer.trainable = True

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
train_history = model.fit_generator(train_gen,
                    steps_per_epoch =math.ceil(train_gen.n/batch_size),
                    epochs = 1,
                    validation_data = val_gen,
                    validation_steps = math.ceil(val_gen.n/batch_size),
                    callbacks=[TQDMNotebookCallback()],
                    verbose=0, 
                    workers = 16)
model.save('Inception_resnetv2_epoch5.h5')

In [ ]:
train_history.history


## Epoch 6

In [ ]:
from keras.optimizers import SGD

In [ ]:
# Learning rate is changed to 0.001
sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
train_history = model.fit_generator(train_gen,
                    steps_per_epoch =math.ceil(train_gen.n/batch_size),
                    epochs = 1,
                    validation_data = val_gen,
                    validation_steps = math.ceil(val_gen.n/batch_size),
                    callbacks=[TQDMNotebookCallback()],
                    verbose=0, 
                    workers = 16)
model.save('Inception_resnetv2_epoch6.h5')

In [ ]:
train_history.history

## Epoch 7

In [9]:
from keras.models import load_model
from keras_tqdm import TQDMNotebookCallback
model = load_model('Inception_resnetv2_epoch6.h5')

In [ ]:
# training last 68 layers 
#for layer in model.layers:
#    layer.trainable = True

In [10]:
#from keras.optimizers import SGD
from keras.optimizers import Adam
adm = Adam(lr=1e-4)
#sgd = SGD(lr=1e-4, decay=1e-4, momentum=0.99, nesterov=True)
model.compile(optimizer=adm, loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
train_history = model.fit_generator(train_gen,
                    steps_per_epoch =math.ceil(train_gen.n/batch_size),
                    epochs = 1,
                    validation_data = val_gen,
                    validation_steps = math.ceil(val_gen.n/batch_size),
                    callbacks=[TQDMNotebookCallback()],
                    verbose=0, 
                    workers = 8)
model.save('Inception_resnetv2_epoch7.h5')

A Jupyter Widget

A Jupyter Widget

In [12]:
train_history.history

{'acc': [0.65477956819753036],
 'loss': [1.5922572999805251],
 'val_acc': [0.62854713684751506],
 'val_loss': [1.8964916627906778]}

## 8

In [13]:
train_history = model.fit_generator(train_gen,
                    steps_per_epoch =math.ceil(train_gen.n/batch_size),
                    epochs = 1,
                    validation_data = val_gen,
                    validation_steps = math.ceil(val_gen.n/batch_size),
                    callbacks=[TQDMNotebookCallback()],
                    verbose=0, 
                    workers = 8)
model.save('Inception_resnetv2_epoch8.h5')

A Jupyter Widget

A Jupyter Widget

In [14]:
train_history.history

{'acc': [0.66236467653672026],
 'loss': [1.5535655244428412],
 'val_acc': [0.6304096572324619],
 'val_loss': [1.89411363175001]}

## 9

In [15]:
train_history = model.fit_generator(train_gen,
                    steps_per_epoch =math.ceil(train_gen.n/batch_size),
                    epochs = 1,
                    validation_data = val_gen,
                    validation_steps = math.ceil(val_gen.n/batch_size),
                    callbacks=[TQDMNotebookCallback()],
                    verbose=0, 
                    workers = 16)
model.save('Inception_resnetv2_epoch9.h5')

A Jupyter Widget

A Jupyter Widget

In [16]:
train_history.history

{'acc': [0.66602726415427138],
 'loss': [1.5329860846091261],
 'val_acc': [0.6309648471672451],
 'val_loss': [1.8863668545084886]}

# 10

In [17]:
#from keras.optimizers import SGD
from keras.optimizers import Adam
adm = Adam(lr=1e-5)
#sgd = SGD(lr=1e-4, decay=1e-4, momentum=0.99, nesterov=True)
model.compile(optimizer=adm, loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
train_history = model.fit_generator(train_gen,
                    steps_per_epoch =math.ceil(train_gen.n/batch_size),
                    epochs = 1,
                    validation_data = val_gen,
                    validation_steps = math.ceil(val_gen.n/batch_size),
                    callbacks=[TQDMNotebookCallback()],
                    verbose=0, 
                    workers = 16)
model.save('Inception_resnetv2_epoch10.h5')

A Jupyter Widget

A Jupyter Widget

In [19]:
train_history.history

{'acc': [0.67101142529447899],
 'loss': [1.5052684286745723],
 'val_acc': [0.63201606079428252],
 'val_loss': [1.8849930250323099]}

# 11

In [9]:
from keras.models import load_model
from keras_tqdm import TQDMNotebookCallback
model = load_model('Inception_resnetv2_epoch10.h5')

In [ ]:
train_history = model.fit_generator(train_gen,
                    steps_per_epoch =math.ceil(train_gen.n/batch_size),
                    epochs = 1,
                    validation_data = val_gen,
                    validation_steps = math.ceil(val_gen.n/batch_size),
                    callbacks=[TQDMNotebookCallback()],
                    verbose=0, 
                    workers = 8)
model.save('Inception_resnetv2_epoch11.h5')

In [11]:
train_bson_file.close()

## Resnet 152
source: https://gist.github.com/flyyufelix/7e2eafb149f72f4d38dd661882c554a6

In [ ]:
from keras.models import Sequential
from keras.optimizers import SGD
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, GlobalAveragePooling2D, ZeroPadding2D, Dropout, Flatten, merge, Reshape, Activation
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras import backend as K

from sklearn.metrics import log_loss

from custom_layers.scale_layer import Scale

In [ ]:
import sys
sys.setrecursionlimit(3000)

In [ ]:
def identity_block(input_tensor, kernel_size, filters, stage, block):
    '''The identity_block is the block that has no conv layer at shortcut
    # Arguments
        input_tensor: input tensor
        kernel_size: defualt 3, the kernel size of middle conv layer at main path
        filters: list of integers, the nb_filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    '''
    eps = 1.1e-5
    nb_filter1, nb_filter2, nb_filter3 = filters
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    scale_name_base = 'scale' + str(stage) + block + '_branch'

    x = Conv2D(nb_filter1, (1, 1), name=conv_name_base + '2a', use_bias=False)(input_tensor)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Scale(axis=bn_axis, name=scale_name_base + '2a')(x)
    x = Activation('relu', name=conv_name_base + '2a_relu')(x)

    x = ZeroPadding2D((1, 1), name=conv_name_base + '2b_zeropadding')(x)
    x = Conv2D(nb_filter2, (kernel_size, kernel_size),
                      name=conv_name_base + '2b', use_bias=False)(x)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Scale(axis=bn_axis, name=scale_name_base + '2b')(x)
    x = Activation('relu', name=conv_name_base + '2b_relu')(x)

    x = Conv2D(nb_filter3, (1, 1), name=conv_name_base + '2c', use_bias=False)(x)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '2c')(x)
    x = Scale(axis=bn_axis, name=scale_name_base + '2c')(x)

    x = merge([x, input_tensor], mode='sum', name='res' + str(stage) + block)
    x = Activation('relu', name='res' + str(stage) + block + '_relu')(x)
    return x

In [ ]:
def conv_block(input_tensor, kernel_size, filters, stage, block, strides=(2, 2)):
    '''conv_block is the block that has a conv layer at shortcut
    # Arguments
        input_tensor: input tensor
        kernel_size: defualt 3, the kernel size of middle conv layer at main path
        filters: list of integers, the nb_filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    Note that from stage 3, the first conv layer at main path is with subsample=(2,2)
    And the shortcut should have subsample=(2,2) as well
    '''
    eps = 1.1e-5
    nb_filter1, nb_filter2, nb_filter3 = filters
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    scale_name_base = 'scale' + str(stage) + block + '_branch'

    x = Conv2D(nb_filter1, (1, 1), subsample=strides,
                      name=conv_name_base + '2a', use_bias=False)(input_tensor)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Scale(axis=bn_axis, name=scale_name_base + '2a')(x)
    x = Activation('relu', name=conv_name_base + '2a_relu')(x)

    x = ZeroPadding2D((1, 1), name=conv_name_base + '2b_zeropadding')(x)
    x = Conv2D(nb_filter2, (kernel_size, kernel_size),
                      name=conv_name_base + '2b', use_bias=False)(x)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Scale(axis=bn_axis, name=scale_name_base + '2b')(x)
    x = Activation('relu', name=conv_name_base + '2b_relu')(x)

    x = Conv2D(nb_filter3, (1, 1), name=conv_name_base + '2c', use_bias=False)(x)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '2c')(x)
    x = Scale(axis=bn_axis, name=scale_name_base + '2c')(x)

    shortcut = Conv2D(nb_filter3, (1, 1), subsample=strides,
                             name=conv_name_base + '1', use_bias=False)(input_tensor)
    shortcut = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '1')(shortcut)
    shortcut = Scale(axis=bn_axis, name=scale_name_base + '1')(shortcut)

    x = merge([x, shortcut], mode='sum', name='res' + str(stage) + block)
    x = Activation('relu', name='res' + str(stage) + block + '_relu')(x)
    return x

In [ ]:
def resnet152_model(img_rows, img_cols, color_type=1, num_classes=None):
    """
    Resnet 152 Model for Keras

    Model Schema and layer naming follow that of the original Caffe implementation
    https://github.com/KaimingHe/deep-residual-networks

    ImageNet Pretrained Weights 
    Theano: https://drive.google.com/file/d/0Byy2AcGyEVxfZHhUT3lWVWxRN28/view?usp=sharing
    TensorFlow: https://drive.google.com/file/d/0Byy2AcGyEVxfeXExMzNNOHpEODg/view?usp=sharing

    Parameters:
      img_rows, img_cols - resolution of inputs
      channel - 1 for grayscale, 3 for color 
      num_classes - number of class labels for our classification task
    """
    eps = 1.1e-5

    # Handle Dimension Ordering for different backends
    global bn_axis
    if K.image_dim_ordering() == 'tf':
        bn_axis = 3
        img_input = Input(shape=(img_rows, img_cols, color_type), name='data')
    else:
        bn_axis = 1
        img_input = Input(shape=(color_type, img_rows, img_cols), name='data')

    x = ZeroPadding2D((3, 3), name='conv1_zeropadding')(img_input)
    x = Conv2D(64, (7, 7), subsample=(2, 2), name='conv1', use_bias=False)(x)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name='bn_conv1')(x)
    x = Scale(axis=bn_axis, name='scale_conv1')(x)
    x = Activation('relu', name='conv1_relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), name='pool1')(x)

    x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')

    x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
    for i in range(1,8):
        x = identity_block(x, 3, [128, 128, 512], stage=3, block='b'+str(i))

    x = conv_block(x, 3, [256, 256, 1024], stage=4, block='a')
    for i in range(1,36):
        x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b'+str(i))

    x = conv_block(x, 3, [512, 512, 2048], stage=5, block='a')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='b')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='c')

    x_fc = GlobalAveragePooling2D(data_format="channels_last", name='global_avg_pool')(x)
    #x_fc = Flatten()(x_fc)
    #x_fc = Dense(1000, activation='softmax', name='fc1000')(x_fc)

    model = Model(img_input, x_fc)

    #if K.image_dim_ordering() == 'th':
    #  # Use pre-trained weights for Theano backend
    #  weights_path = 'imagenet_models/resnet152_weights_th.h5'
    #else:
    #  # Use pre-trained weights for Tensorflow backend
    #  weights_path = 'imagenet_models/resnet152_weights_tf.h5'
    weights_path = 'imagenet_models/resnet152_weights_tf.h5'
    model.load_weights(weights_path, by_name=True)

    # Truncate and replace softmax layer for transfer learning
    # Cannot use model.layers.pop() since model is not of Sequential() type
    # The method below works since pre-trained weights are stored in layers but not in the model
    x_newfc = GlobalAveragePooling2D(data_format="channels_last", name='global_avg_pool')(x)
    #x_newfc = Flatten()(x_newfc)
    x_newfc = Dense(num_classes, activation='softmax', name='fc8')(x_newfc)

    model = Model(img_input, x_newfc)
    
    # Learning rate is changed to 0.001
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

## Training

In [ ]:
img_rows=180
img_cols=180
img_channels=3
num_classes=5270
batch_size = 128

In [ ]:
# Load our model
model = resnet152_model(img_rows, img_cols, img_channels, num_classes)


In [ ]:
model.summary()

### Training the model

In [ ]:
# To train the model:
train_history = model.fit_generator(train_gen,
                    steps_per_epoch =math.ceil(train_gen.n/batch_size),
                    epochs = 1,
                    validation_data = val_gen,
                    validation_steps = math.ceil(val_gen.n/batch_size),
                    workers = 18)
model.save('resnet152.h5')

### Test